## Step 1: Importing the necessary libaries

In [ ]:
import os
import tensorflow as tf
from google.colab import files
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt

Files_save_path = '/content/drive/MyDrive/<path>'
os.makedirs(Files_save_path, exist_ok=True)

## Step 2: Load the Dataset into TensorFlow


In [ ]:
# PATH TO THE DATASET
dataset_path = '/content/drive/MyDrive/<path>'

# TRAINING DATASET V2 (80% OF DATA)
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(128, 128),
    batch_size=32
)

# VALIDATION DATASET V2 (20% OF DATA)
val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(128, 128),
    batch_size=34
)

# CLASS NAMES
class_names = train_ds.class_names
print(f"Class Names: {class_names}")

## Step 3: Visualizing the images



In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):  # IT WILL TAKE ONE BATCH
    for i in range(9):  # SHOW 9 IMAGES
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # CONVERT TO PROPER FORMAT
        plt.title(class_names[labels[i]])  # ADD LABEL
        plt.axis("off")
    break
plt.show()

## Step 4: Normalize the data


In [ ]:
# NORMALIZE PIXEL VALUES (DIVIDE BY 255)
train_ds = train_ds.map(lambda x, y: (x / 255.0, y)) # X REPRESENTS IMAGE DATA, AND Y REPRESENTS LABELS
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

## Step 5: Build a CNN Model

In [ ]:
#BUILD THE MODEL
model = tf.keras.Sequential([
    # CONVOLUTION LAYER
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    # POOLING LAYER
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # CONVERT FEATURE MAPS TO 1D
    tf.keras.layers.Flatten(),
    # FULLY CONNECTED LAYER
    tf.keras.layers.Dense(128, activation='relu'),
    # OUTPUT LAYER (SOFTMAX FOR MULTI-CLASS)
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

#COMPILE THE MODEL
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#DISPLAY THE MODEL SUMMARY
model.summary()

## Step 6: Train the model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)

model.save('/content/drive/MyDrive/<path>')
files.download('/content/drive/MyDrive/<path>')

## Step 7: Plot Training Results

In [ ]:
import os
import matplotlib.pyplot as plt

fig_acc = plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title("Accuracy over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
fig_acc.savefig(os.path.join(Files_save_path, 'Accuracy_vs_Epochs.png'), dpi=300)
fig_acc.savefig(os.path.join(Files_save_path, 'Accuracy_vs_Epochs.pdf'), dpi=300)
plt.close(fig_acc)  # CLOSE TO AVOID OVERLAP IN NEXT PLOT

# --- LOSS PLOT ---
fig_loss = plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
fig_loss.savefig(os.path.join(Files_save_path, 'Loss_vs_Epochs.png'), dpi=300)
fig_loss.savefig(os.path.join(Files_save_path, 'Loss_vs_Epochs.pdf'), dpi=300)
plt.close(fig_loss)

## Step 8: Confusion Matrix & Heatmap


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# GENERATING TRUE AND PREDICTED LABELS
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# CREATING CONFUSION MATRIX
cm = confusion_matrix(y_true, y_pred)
fig = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# SAVING BEFORE PLT.SHOW()
fig.savefig(os.path.join(Files_save_path, 'confusion_matrix.png'), dpi=300)
fig.savefig(os.path.join(Files_save_path, 'confusion_matrix.pdf'), dpi=300)

plt.show()

## Step 9: Classification Report as Bar Plots

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# GET PRECISION, RECALL, F1, AND SUPPORT
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, labels=range(len(class_names)))

x = np.arange(len(class_names))
width = 0.25

fig = plt.figure(figsize=(12, 6))
plt.bar(x - width, precision, width, label='Precision')
plt.bar(x, recall, width, label='Recall')
plt.bar(x + width, f1, width, label='F1-score')

plt.ylabel('Score')
plt.title('Classification Report Metrics')
plt.xticks(x, class_names, rotation=45)
plt.ylim([0, 1.1])
plt.legend()
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# ✅ SAVE THE FIGURE BEFORE PLT.SHOW()
fig.savefig(os.path.join(Files_save_path, 'classification_report.png'), dpi=300)
fig.savefig(os.path.join(Files_save_path, 'classification_report.pdf'), dpi=300)

plt.show()

## Step 10: Test the Model


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# LOAD AN IMAGE
img_path = '/content/drive/MyDrive/Colab Notebooks/Research_Project/datasets/waste_dataset_v2/glass/glass5.jpg'
img = image.load_img(img_path, target_size=(128, 128))

# CONVERT IMAGE TO ARRAY AND NORMALIZE
img_array = np.expand_dims(np.array(img) / 255.0, axis=0)

# PREDICT THE CLASS
prediction = model.predict(img_array)
predicted_class = class_names[np.argmax(prediction)]

print(f"Predicted Class: {predicted_class}")